In [1]:
def get_innerlinks(url, lock, i, link_path):  #找所有連結
    #call global var
    global HEADER
    global alinks
    global page_links
    count = 5             # retry 5 times
    
    while count:
        try:
            res = requests.get(url, headers=HEADER)
            soup = BeautifulSoup(res.text, 'lxml')
            links = soup.select('div.jbInfo > div > h3 > a')
            if res.status_code == 200 and len(links) != 0:
                for link in links:  
                    page_links.append('https:' + str(link['href'])) #href put in innerURL function           
                print('Get {} links from page {}'.format(len(page_links), i))              
                with lock:
                    with open(link_path, 'a') as fw:
                        alinks += page_links
                        fw.write('\n'.join(page_links))
                        fw.write('\n')
                break
            else:
                print ('Cannot retrieve links from page {} Retry: {} '.format(i, 6 - count))
                count -= 1
                time.sleep(1)
        except Exception as e:
            print ('Cannot retrieve links from page {} Retry: {} Error: {}'.format(i, 6 - count, e))
            count -= 1
            time.sleep(1)
            
    if count == 0:
        print('[Error] Page {:2} failed'.format(i))

In [2]:
def grab_content(url,lock,idx): #一個用來分析內頁的function
    
    global page_links
    global HEADER
    global wc

    count = 5
    while count:
        try:
            res2 = requests.get(url, headers=HEADER)
            if res2.status_code == 200:
                soup = BeautifulSoup(res2.text, 'lxml')
                dds = soup.select('dl.dataList > dd')
                text = ''
                for dd in dds:
                    for a in dd.find_all('a'):
                        a.decompose()
                    for img in dd.find_all('img'):
                        img.decompose()
                    text += dd.text
                words = list(set(re.findall('object c|visual basic|[A-Za-z.+#]+', text, re.IGNORECASE)))
                with lock:      
                    for lan in words: 
                        if lan in wc:  
                            wc[lan] += 1
                        else:
                            wc[lan] = 1
                if idx % 200 == 0:
                    print('{} have been completed'.format(idx))
                break
            else:
                print ('Cannot retrieve contentfrom page {} Retry: {} '.format(idx, 6 - count))
                count -= 1
                time.sleep(0.5)
        except Exception as e:
            print ('Cannot retrieve contentfrom page {} Retry: {} Error; {} '.format(idx, 6 - count), e)
            count -= 1
            time.sleep(0.5)
    if count == 0:
        print('Retrieve contetn failed page {} '.format(idx))

In [3]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
import time
import threading
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import re

In [ ]:
#mainThread  
#Get pageNum and Use thread to send pageLink then Use thread to send innerLink then write in csv

HOST = 'https://www.1111.com.tw'
HEADER = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.96 Safari/537.36'}

# global variable
thread_list=[]
alinks = []
page_links = []
wc = Counter()
lock = threading.Lock()
link_path = './link_1111.csv'
#*********************************************************************************************************#

#Get total pages
url =  HOST + '/job-bank/job-index.asp?si=1&d0=140400,140200,140300&fs=1&ps=100&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'lxml')
page = int(soup.select_one('div.pagedata').text.split('/')[1].split('頁')[0]) + 1

# Get all links from all pages
for i in range(1,page):
    url =  HOST + '/job-bank/job-index.asp?si=1&d0=140400,140200,140300&fs=1&ps=100&page={}'.format(i)
    th = threading.Thread(target=get_innerlinks, args=(url, lock, i, link_path))
    th.start()
    time.sleep(0.3)
    thread_list.append(th)
else:                                       #  not error
    print('[Debug] All threads are started')

# Join all threads
for thread in thread_list:
    thread.join()

thread_list2 = []

# Not applicable to this case !!
# Get all content from alllinks
# with ThreadPoolExecutor(max_workers=5) as executor:
#     for idx, link in enumerate(alinks):
#         executor.submit(grab_content, link, lock, idx) 

for idx, link in enumerate(alinks):
    th = threading.Thread(target=grab_content, args=(link, lock, idx ))
    th.start()
    time.sleep(0.2)
    thread_list2.append(th)
for thread in thread_list2:
    thread.join()
        
# Wirte results to file
with open('./1111.csv', 'w') as fw2:
    for lang, counts in wc.most_common():
        fw2.write('{},{}\n'.format(lang,counts))
#輸出成json檔
with open('./1111.json', 'w') as fw3:
    for lang, counts in wc.most_common():
        json.dump(language, f)


[Error] Page  4 failed
Cannot retrieve links from page 5 Retry: 5 Error: name 'page_links' is not defined
Get 100 links from page 16
Get 200 links from page 27
Get 300 links from page 28
[Error] Page  5 failed
Get 400 links from page 13
Get 500 links from page 19
Get 600 links from page 18
Get 700 links from page 20
Get 800 links from page 9
Get 900 links from page 17
Get 1000 links from page 21
Get 1100 links from page 10
Get 1200 links from page 11
Get 1305 links from page 1
Get 1405 links from page 22
Get 1505 links from page 7
Get 1605 links from page 23
Get 1705 links from page 12
Get 1805 links from page 14
Get 1905 links from page 29
Get 2005 links from page 24
Get 2105 links from page 6
Get 2205 links from page 25
Get 2305 links from page 15
Get 2405 links from page 26
Get 2505 links from page 8
Get 2605 links from page 30
Get 2710 links from page 2
Get 2815 links from page 1
Get 2920 links from page 2
Get 3020 links from page 3
Get 3120 links from page 4
Get 3220 links from pa

Exception in thread Thread-309:
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/socket.py", line 743, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/ubuntu/

400 have been completed


Exception in thread Thread-722:
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/socket.py", line 743, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/ubuntu/

800 have been completed


Exception in thread Thread-436:
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connection.py", line 141, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/connection.py", line 83, in create_connection
    raise err
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [Errno 110] Connection timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/conn

1000 have been completed


Exception in thread Thread-461:
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 386, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/site-packages/requests/packages/urllib3/connectionpool.py", line 382, in _make_request
    httplib_response = conn.getresponse()
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/home/ubuntu/.pyenv/versions/3.6.1/lib/python3.6/http/client.py", line 266, in _read_status
    raise R

In [ ]:
wc.most_common()